In [1]:
from langchain_community.embeddings import HuggingFaceEmbeddings

model_name="sentence-transformers/all-MiniLM-l6-v2"
embedding = HuggingFaceEmbeddings(model_name=model_name)
embedding

/tmp/ipykernel_9567/2373350834.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(model_name=model_name)
/home/ahmed/miniconda3/envs/chatapp/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False, 'architecture': 'BertModel'})
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-l6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [1]:
from dotenv import load_dotenv

import os

load_dotenv()

True

## LLM loader

In [2]:

from langchain_groq import ChatGroq
groq_api_key = os.getenv('GROQ_API_KEY')

llm = ChatGroq(
    model="llama-3.1-8b-instant",
    api_key=groq_api_key,
    max_tokens=500
)


class Groqllm:
    def __init__(self, model="llama-3.1-8b-instant", api_key=None, max_tokens=500):
        self.model = model
        self.api_key = api_key
        self.max_tokens = max_tokens
    
    def call(self):

        model = ChatGroq(
            model=self.model,
            api_key=self.api_key,
            max_tokens=self.max_tokens
        )
        return model
    
    def invoke(self,query:str):
        llm = self.call()
        response = llm.invoke(query)

        return response

/home/ahmed/miniconda3/envs/chatapp/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
llm = Groqllm(api_key=groq_api_key)
model = llm.call()
model.invoke("Hello, LangChain with Groq!")

AIMessage(content="LangChain is an open-source library for building conversational AI applications using Python. It provides a range of tools and features to help developers integrate and use large language models (LLMs) in their projects.\n\nGroq, on the other hand, is a company that specializes in building high-performance AI chips, and they also provide a cloud-based service for running machine learning models, including large language models.\n\nCombining LangChain with Groq's technology could potentially enable the development of highly efficient and scalable conversational AI systems. LangChain's library could be used to build and integrate conversational AI models, while Groq's technology could be used to run these models on high-performance hardware, enabling faster and more accurate responses.\n\nSome potential use cases for LangChain with Groq could include:\n\n1. **Conversational AI for customer service**: Using LangChain to build conversational AI models that can understand

## Document Loader

In [ ]:
# from langchain_classic.document_loaders import PyPDFLoader

# loader = PyPDFLoader("project/data/MachineLearning.pdf")
# pages = loader.load_and_split()

# for i in range(len(pages)):
#     print(pages[i].page_content)

In [21]:
from langchain_classic.document_loaders import PyPDFLoader

loader = PyPDFLoader("project/data/MachineLearning.pdf")
docs = loader.load()

In [ ]:
for i, page in enumerate(docs):
    print(f"Page {i}:")
    print(page.page_content[:200])
    print("-" * 50)


In [3]:
from langchain_classic.document_loaders import PyPDFLoader


class DocumentProcessor:
    def __init__(self, file_path:str):
        self.file_path = file_path
        self.loader = PyPDFLoader(self.file_path)
    
    def load_documents(self):
        documents = self.loader.load()
        return documents
    
    def display_pages(self,documents,idx=5,num_chars=200):
        for idx,page in enumerate(documents[:idx]):
            print(f"Page: {idx}")
            print("Page_content:",page.page_content[:num_chars])
        
        

In [4]:
loader = DocumentProcessor("project/data/MachineLearning.pdf")
docs = loader.load_documents()
loader.display_pages(docs,idx=5,num_chars=200)

Page: 0
Page_content: The
Hundred-
Page
Machine
Learning
Book
Andriy Burkov
Page: 1
Page_content: “All models are wrong, but some are useful.”
— George Box
The book is distributed on the “read ﬁrst, buy later” principle.
Andriy Burkov The Hundred-Page Machine Learning Book - Draft
Page: 2
Page_content: Preface
Let’s start by telling the truth: machines don’t learn. What a typical “learning machine”
does, is ﬁnding a mathematical formula, which, when applied to a collection of inputs (called
“trainin
Page: 3
Page_content: in a speciﬁc topic covered in the book and want to know more, most sections have a QR
code. By scanning one of those QR codes with your phone, you will get a link to a page on
the book’s companion wik
Page: 4
Page_content: TheHundred-Page
MachineLearning
Book
Andriy Burkov


## Split & Chunks

In [5]:
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter


class SplitterDocumentProcessor:
    def __init__(self,documents,chunk_size=1000,chunk_overlap=200):
        self.documents = documents
        self.chunk_size = chunk_size
        self.chunk_overlap = chunk_overlap
        
    def split_doc(self):
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=self.chunk_size,
            chunk_overlap=self.chunk_overlap
        )

        all_chunks = []
        for doc in self.documents:
            print("Original document length:", len(doc.page_content))
            chunks = text_splitter.split_text(doc.page_content)

            for idx,chunk in enumerate(chunks):
                #print(f"Chunk {idx} length:", len(chunk))
                chunk_doc = Document(page_content=chunk)
                all_chunks.append(chunk_doc)
        
        return all_chunks

In [ ]:
split = SplitterDocumentProcessor(documents=docs, chunk_size=1000, chunk_overlap=200)
chunk = split.split_doc()

Original document length: 53
Original document length: 183
Original document length: 2816
Original document length: 1404
Original document length: 50
Original document length: 184
Original document length: 2619
Original document length: 2738
Original document length: 2958
Original document length: 2338
Original document length: 1552
Original document length: 3319
Original document length: 559
Original document length: 50
Original document length: 184
Original document length: 1470
Original document length: 1841
Original document length: 1716
Original document length: 1836
Original document length: 820
Original document length: 2450
Original document length: 1586
Original document length: 1762
Original document length: 1976
Original document length: 2535
Original document length: 1798
Original document length: 50
Original document length: 184
Original document length: 2378
Original document length: 1438
Original document length: 3516
Original document length: 1462
Original document leng

In [12]:
len(chunk)

404

## Embedding

In [ ]:
from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings

model_name="sentence-transformers/all-MiniLM-l6-v2"
embedding = HuggingFaceEmbeddings(model_name=model_name)
embedding

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from sentence_transformers import SentenceTransformer
from typing import List,Any

class EmbeddingPipeline:
    def __init__(self,model_name:str="sentence-transformers/all-MiniLM-l6-v2",
                 chunk_size:int=1000,chunk_overlap:int=200):
        
        self.model = SentenceTransformer(model_name)
        #self.model = HuggingFaceEmbeddings(model_name)
        self.chunk_size = chunk_size
        self.chunk_overlap = chunk_overlap
    
    def split_doc(self,documents:List[Any])-> List[Any]:
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=self.chunk_size,
            chunk_overlap=self.chunk_overlap,
            separators=["\n\n", "\n", " ", ""]
        )

        text = []
        for doc in documents:
            print("Original document length:", len(doc.page_content))
            chunks = text_splitter.split_text(doc.page_content)

            for idx,chunk in enumerate(chunks):
                #print(f"Chunk {idx} length:", len(chunk))
                chunk_doc = Document(page_content=chunk)
                text.append(chunk_doc)
        
        return text
    
    def embed_chunk(self,texts:List[Any])->List[Any]:
        embedding = self.model.encode(texts,show_progress_bar=True)

        return embedding
    

def embed_model(model_name:str="sentence-transformers/all-MiniLM-l6-v2"):
    embed_model = HuggingFaceEmbeddings(model_name=model_name)

    return embed_model

In [ ]:
loader = DocumentProcessor("project/data/MachineLearning.pdf")
docs = loader.load_documents()

emb_pipe = EmbeddingPipeline()
texts = emb_pipe.split_doc(documents=docs)

In [14]:
texts

[Document(metadata={}, page_content='The\nHundred-\nPage\nMachine\nLearning\nBook\nAndriy Burkov'),
 Document(metadata={}, page_content='“All models are wrong, but some are useful.”\n— George Box\nThe book is distributed on the “read ﬁrst, buy later” principle.\nAndriy Burkov The Hundred-Page Machine Learning Book - Draft'),
 Document(metadata={}, page_content='Preface\nLet’s start by telling the truth: machines don’t learn. What a typical “learning machine”\ndoes, is ﬁnding a mathematical formula, which, when applied to a collection of inputs (called\n“training data”), produces the desired outputs. This mathematical formula also generates the\ncorrect outputs for most other inputs (distinct from the training data) on the condition that\nthose inputs come from the same or a similar statistical distribution as the one the training\ndata was drawn from.\nWhy isn’t that learning? Because if you slightly distort the inputs, the output is very likely\nto become completely wrong. It’s not ho

In [10]:
embedding = emb_pipe.embed_chunk(texts=[text.page_content for text in texts])

Batches:   0%|          | 0/13 [00:00<?, ?it/s]

Batches: 100%|██████████| 13/13 [00:27<00:00,  2.12s/it]


In [11]:
embedding

array([[-0.10072465, -0.04188386, -0.04252208, ...,  0.08731069,
        -0.02709884, -0.01210178],
       [-0.05580553, -0.06041165,  0.02039663, ...,  0.0624044 ,
        -0.03595995,  0.02714251],
       [ 0.00869817, -0.05804476,  0.06154076, ...,  0.06583741,
        -0.01441926, -0.03575968],
       ...,
       [-0.127216  ,  0.01841581, -0.0014909 , ...,  0.04112707,
        -0.03507386, -0.00153942],
       [-0.09881318,  0.01418942, -0.03938656, ...,  0.01772543,
        -0.01342625,  0.03951202],
       [-0.04323079, -0.09646977,  0.00910576, ...,  0.01894681,
        -0.02291583,  0.03039983]], shape=(404, 384), dtype=float32)

In [12]:
loader = DocumentProcessor("project/data/MachineLearning.pdf")
docs = loader.load_documents()

emb_model = embed_model()
text = emb_model.embed_documents(texts=[doc.page_content for doc in docs])
text

[[-0.10072465240955353,
  -0.04188385605812073,
  -0.04252208024263382,
  -0.052967268973588943,
  0.007949753664433956,
  0.045769836753606796,
  0.022930219769477844,
  -0.02619156800210476,
  -0.08693287521600723,
  -0.02008146606385708,
  -0.03408456966280937,
  0.07232068479061127,
  0.07357002794742584,
  -0.0686749666929245,
  -0.09256329387426376,
  0.04259215295314789,
  -0.07571352273225784,
  0.04704556241631508,
  -0.025384506210684776,
  -0.07132858037948608,
  -0.03499959036707878,
  0.07699738442897797,
  0.03366424888372421,
  0.02023916319012642,
  -0.002521730260923505,
  -0.027795812115073204,
  0.02517378330230713,
  -0.023431530222296715,
  0.015965193510055542,
  -0.014361086301505566,
  -0.013803600333631039,
  0.00854815449565649,
  0.004116094671189785,
  0.008286625146865845,
  -0.003425405826419592,
  0.011808748356997967,
  -0.005855395924299955,
  0.008493156172335148,
  0.026112917810678482,
  0.08818362653255463,
  0.03639621287584305,
  -0.02501982077956

In [13]:
emb_model.embed_query("What is machine learning?")

[-0.01995455101132393,
 0.009877976030111313,
 0.010249646380543709,
 0.029553720727562904,
 0.027186432853341103,
 -0.019296549260616302,
 -0.024129603058099747,
 -0.037735745310783386,
 -0.04105420038104057,
 -0.0014749857364222407,
 -0.07607674598693848,
 0.03687206655740738,
 0.053723812103271484,
 -0.07053519785404205,
 -0.08609344065189362,
 0.021006854251027107,
 -0.04226333275437355,
 0.028448956087231636,
 -0.03837535157799721,
 -0.06591921299695969,
 0.01394961029291153,
 0.012275398708879948,
 -0.06821971386671066,
 0.028625089675188065,
 0.014761177822947502,
 0.03223147615790367,
 0.008694251999258995,
 0.013062091544270515,
 -0.015726696699857712,
 0.0188965555280447,
 0.020023856312036514,
 -0.039297085255384445,
 0.01564721018075943,
 0.03339797630906105,
 -0.07339102029800415,
 0.05935779958963394,
 -0.012385822832584381,
 0.02458464726805687,
 0.020107122138142586,
 -0.020254533737897873,
 -0.050247687846422195,
 -0.09044115990400314,
 -0.0153743801638484,
 -0.0036123

## Vactore store

In [ ]:
from langchain_community.vectorstores import FAISS


class VectorStorePipeline:

    def __init__(self):
        pass

    def store(self,texts:List[Any]):
        #doc = [doc.page_content for doc in docs]
        vectore_store = FAISS.from_documents(texts,emb_model)
        retriver = vectore_store.as_retriever(search_type="similarity", search_kwargs={"k": 3})
    
        return retriver
    
    def search(self,retriver,query):
        result = retriver.vectorstore.similarity_search_with_score(query, k=4)

        return result
        

In [31]:
vec_store = VectorStorePipeline()
retriver = vec_store.store(texts)

query = 'Whats machine learning'
response = vec_store.search(retriver,query)


In [32]:
response

[(Document(id='f87f8b26-957d-4ee1-b885-1ea08319d650', metadata={}, page_content='1 Introduction\n1.1 What is Machine Learning\nMachine learning is a subﬁeld of computer science that is concerned with building algorithms\nwhich, to be useful, rely on a collection of examples of some phenomenon. These examples\ncan come from nature, be handcrafted by humans or generated by another algorithm.\nMachine learning can also be deﬁned as the process of solving a practical problem by 1)\ngathering a dataset, and 2) algorithmically building a statistical model based on that dataset.\nThat statistical model is assumed to be used somehow to solve the practical problem.\nTo save keystrokes, I use the terms “learning” and “machine learning” interchangeably.\n1.2 Types of Learning\nLearning can be supervised, semi-supervised, unsupervised and reinforcement.\n1.2.1 Supervised Learning\nIn supervised learning1,t h edataset is the collection oflabeled examples{(xi,y i)}N\ni=1.\nEach elementxi among N is 

In [33]:
response[0][0].page_content

'1 Introduction\n1.1 What is Machine Learning\nMachine learning is a subﬁeld of computer science that is concerned with building algorithms\nwhich, to be useful, rely on a collection of examples of some phenomenon. These examples\ncan come from nature, be handcrafted by humans or generated by another algorithm.\nMachine learning can also be deﬁned as the process of solving a practical problem by 1)\ngathering a dataset, and 2) algorithmically building a statistical model based on that dataset.\nThat statistical model is assumed to be used somehow to solve the practical problem.\nTo save keystrokes, I use the terms “learning” and “machine learning” interchangeably.\n1.2 Types of Learning\nLearning can be supervised, semi-supervised, unsupervised and reinforcement.\n1.2.1 Supervised Learning\nIn supervised learning1,t h edataset is the collection oflabeled examples{(xi,y i)}N\ni=1.\nEach elementxi among N is called afeature vector. A feature vector is a vector in which'

## prompt

In [42]:
from langchain_core.prompts import PromptTemplate

def prompt():
    prompt_template = """You are an assistant for question-answering tasks.
    Use the following pieces of retrieved context to answer the question.
    If you don't know the answer, say that you don't know.
    Use three sentences maximum and keep the answer concise.

    Question: {input}
    Context: {context}
    Answer:
    """

    prompt = PromptTemplate.from_template(prompt_template)

    return prompt



In [43]:
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_classic.chains.retrieval import create_retrieval_chain

groq = Groqllm()
llm = groq.call()
prompt = prompt()


combine_docs_chain = create_stuff_documents_chain(llm, prompt)

result = create_retrieval_chain(retriver, combine_docs_chain)
result

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7f70b9385130>, search_kwargs={'k': 3}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | PromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks.\n    Use the following pieces of retrieved context to answer the question.\n    If you don't know the answer, say that you don't know.\n    Use three sentences maximum and keep the answer concise.\n\n    Question

In [45]:
print(result.input_schema.model_json_schema())

{'properties': {'input': {'title': 'Input'}}, 'required': ['input'], 'title': 'RunnableParallel<context>Input', 'type': 'object'}


In [47]:
response = result.invoke({
    "input": "Whats machine learning"
})

print(response["answer"])

Machine learning is a subfield of computer science that involves building algorithms using a collection of examples to solve practical problems. It's a process of gathering a dataset and algorithmically building a statistical model based on that dataset. This statistical model is then used to solve the practical problem.


In [48]:
response

{'input': 'Whats machine learning',
 'context': [Document(id='f87f8b26-957d-4ee1-b885-1ea08319d650', metadata={}, page_content='1 Introduction\n1.1 What is Machine Learning\nMachine learning is a subﬁeld of computer science that is concerned with building algorithms\nwhich, to be useful, rely on a collection of examples of some phenomenon. These examples\ncan come from nature, be handcrafted by humans or generated by another algorithm.\nMachine learning can also be deﬁned as the process of solving a practical problem by 1)\ngathering a dataset, and 2) algorithmically building a statistical model based on that dataset.\nThat statistical model is assumed to be used somehow to solve the practical problem.\nTo save keystrokes, I use the terms “learning” and “machine learning” interchangeably.\n1.2 Types of Learning\nLearning can be supervised, semi-supervised, unsupervised and reinforcement.\n1.2.1 Supervised Learning\nIn supervised learning1,t h edataset is the collection oflabeled exampl